![nlp](https://dv-website.s3.amazonaws.com/uploads/2018/11/what_is_NLP.jpg)

# Tutorial Spark NLP
### Megadados 2019 - Alessandra Blücher

# Por que eu não posso programar em português?

Existem milhares de linguagens diferentes de programação. Baixo ou alto nível, orientadas a objeto ou procedurais, mais ou menos intuitivas. Com uma variedade tão grande é possível encontrar uma para se adequar às mais diversas demandas. Entretanto, nenhuma dessas linguagens mencionadas pode ser considerada `natural`, como os dialetos que conhecemos.

`Quais são as diferenças entre linguagens de programação e linguagens naturais que tornam impossível a ideia de programar com elas?`

A resposta simples é que as linguagens naturais são inerentemente ambíguas. Um dos passos que o computador executa para entender determinada instrução é o parceamento da linguagem. Nós humanos também temos o mesmo processo de parceamento para compreender o português ou qualquer outra linguagem natural. Passivamente nós lidamos com problemas da linguagem natural que o computador não é capaz de lidar com tanta facilidade. Observe os exemplos a seguir de situações onde a linguagem natural possui algum aspecto que é um empecilho para o processamento:

1. Ambiguidade

    "Ana encontrou o gerente da loja com o seu irmão." A frase é ambígua pois não se sabe se o irmão é de Ana ou do gerente. Esse tipo de situação é inexistente em linguagens de programação. Diferentes Processamentos de Linguagem Natural podem lidar com esse problema de algumas maneiras, uma delas é um parser probabilístico, que avalia qual das hipóteses é mais provável de ser o que o autor realmente quis dizer.
    
    
2. Correferência

    "O ministro da educação atual pediu por uma pesquisa do índice de analfabetos acima de 60 anos no Brasil. Ele estima que o número deve ter diminuído em 27%." O comportamento presente nessa frase que é inexistente em linguagens de programação é a correferência apresentada pelo pronome 'ele' que se refere ao 'ministro da educação'.


3. Polissemia
    
    "Cabo":
    1. Posto militar; B. Acidente geográfico; C. Cabo da vassoura

   "Banco":
    1. Instituição comercial financeira; B. Assento

   "Manga":
    1. Parte da roupa; B. Fruta
    
    Esses são alguns exemplos de palavras polissêmicas, ou seja, palavras com múltiplos significados dependendo do contexto. Onde seria simples para nós distinguirmos qual sentido o contexto requer, fica evidente que para o computador poder fazer o mesmo é preciso um processamento específico da linguagem natural.
    



###### Curiosidade:
`Lojban` é uma linguagem natural construída que não possui nenhuma ambiguidade. Teoricamente ela pode ser um dialeto comum e uma linguagem de programação ao mesmo tempo (alguns teste já foram feitos usando Lojban!).
Para mais informações acesse: https://en.wikipedia.org/wiki/Lojban

Agora que você entendeu o `porquê` do processamento de linguagem natural, vamos entender mais sobre o `como`.

###### Atenção! 


A partir desse momento vamos começar a efetivamente falar sobre e usar a ferramenta Spark NPL. As pipelines disponíveis pelo John Snow Labs possuem três idiomas: Inglês, Francês e Italiano.
Nós utilizaremos `Inglês`.

# Instalando

Para fazer a instalação do Spark NPL siga as instruções oficiais oferecidas no site:

https://nlp.johnsnowlabs.com/docs/en/install

# Introduzindo Spark NLP

Spark NLP é uma biblioteca de processamento de linguagem natural open source, construída em cima do Apache Spark e Spark ML. A biblioteca produzida pelo John Snow Labs fornece annotations simples, eficazes e precisas para pipelines de Machine Learning, enquanto ela é facilmente escalável em ambientes distribuídos.

A tabela abaixo mostra a comparação das funcionalidades do Spark NPL em comparação com outras bibliotecas:


![functionalities](https://blog.dominodatalab.com/wp-content/uploads/2019/04/Pacific-AI-Table-1-cx-Sep-2019.png)

Outro aspecto excepecional do Spark NLP é a escalabilidade. A construção nativa no Apache Spark ML permite que o Spark NLP seja dimensionado em qualquer cluster Spark, no local ou em qualquer provedor de nuvem. As acelerações (speed ups) são otimizadas graças ao planejamento e cache de execução distribuída do Spark, que foram testados em praticamente qualquer plataforma de armazenamento e computação atual.

# Observando a Pipeline

Para compreender melhor o uso do Spark NLP vamos começar com uma demonstração simples do uso da Pipeline pré treinada `explain_document`.

Uma pipeline é a lista de etapas do processamento do texto. No caso do explain_document temos as seguintes etapas:

 * DocumentAssembler
 * SentenceDetector
 * Tokenizer
 * Lemmatizer
 * Stemmer
 * Part of Speech
 * SpellChecker (Norvig)

In [1]:
import sys
import time

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher

#### Criando uma sessão Spark

In [2]:
spark = sparknlp.start()
print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Spark NLP version
2.2.1
Apache Spark version


'2.4.4'

#### Esse será nosso documento de teste para vizualizar as etapas 

In [3]:
testDoc = [
"life-changing It is change, continuing change, inivitable change, \
that is the  dominant factor in society today. \
No sensinble decision can be made any longer without \
taking into account not only the world as it is, \
but the world as it will be. "
]

`Não se assuste com os erros gramaticais, nós iremos lidar com eles mais tarde!`

#### Declarando nossa pipeline pré treinada

In [4]:
# Carrega a versão em inglês da pipeline explain_document_dl,
# os modelos pré treinado e todas suas dependencias - cache local
pipeline = PretrainedPipeline('explain_document_ml', lang='en')

explain_document_ml download started this may take some time.
Approx size to download 9,4 MB
[OK!]


In [5]:
#annotate() o algoritmo de cada estágio da pipelina
result = pipeline.annotate(testDoc) 

#### Vamos observar os resultados para enteder melhor o resultado de cada etapa do processamento.

In [6]:
[content['sentence'] for content in result]

[['life-changing It is change, continuing change, inivitable change, that is the  dominant factor in society today.',
  'No sensinble decision can be made any longer without taking into account not only the world as it is, but the world as it will be.']]

#### Agora vamos de frases para tokens

In [7]:
[content['token'] for content in result]

[['life-changing',
  'It',
  'is',
  'change',
  ',',
  'continuing',
  'change',
  ',',
  'inivitable',
  'change',
  ',',
  'that',
  'is',
  'the',
  'dominant',
  'factor',
  'in',
  'society',
  'today',
  '.',
  'No',
  'sensinble',
  'decision',
  'can',
  'be',
  'made',
  'any',
  'longer',
  'without',
  'taking',
  'into',
  'account',
  'not',
  'only',
  'the',
  'world',
  'as',
  'it',
  'is',
  ',',
  'but',
  'the',
  'world',
  'as',
  'it',
  'will',
  'be',
  '.']]

# Spell Checking & Correction

#### Reparou em alguns erros de ortografia? Vamos ver se a pipeline corrigiu!

Preste atenção em `inivitable` e `sensinble`:

In [8]:
[content['checked'] for content in result]

[['lifechanging',
  'It',
  'is',
  'change',
  ',',
  'continuing',
  'change',
  ',',
  'inevitable',
  'change',
  ',',
  'that',
  'is',
  'the',
  'dominant',
  'factor',
  'in',
  'society',
  'today',
  '.',
  'No',
  'sensible',
  'decision',
  'can',
  'be',
  'made',
  'any',
  'longer',
  'without',
  'taking',
  'into',
  'account',
  'not',
  'only',
  'the',
  'world',
  'as',
  'it',
  'is',
  ',',
  'but',
  'the',
  'world',
  'as',
  'it',
  'will',
  'be',
  '.']]

A pipeline que usamos já realiza a correção da ortografia, mas é possível treinar ela de três formas diferentes:
* Norvig Approach:
    - retorna token e auto corrige baseado num dado dicionário
* Symmetric Delete:
    - usa métricas de distância para achar possíveis palavras
* Context Aware:
    - mais preciso, julga as palavras no contexto
    - baseado em deep learning

#### Vamos ver os lemmas

Lemmas o resultado de um processo de lematização, que agrupo palavras que estejam relacionadas a outras e ajusta as flexões das palavras para interpretá-las melhor.

In [9]:
[content['lemma'] for content in result]

[['lifechanging',
  'It',
  'be',
  'change',
  ',',
  'continue',
  'change',
  ',',
  'inevitable',
  'change',
  ',',
  'that',
  'be',
  'the',
  'dominant',
  'factor',
  'in',
  'society',
  'today',
  '.',
  'No',
  'sensible',
  'decision',
  'can',
  'be',
  'make',
  'any',
  'long',
  'without',
  'take',
  'into',
  'account',
  'not',
  'only',
  'the',
  'world',
  'as',
  'it',
  'be',
  ',',
  'but',
  'the',
  'world',
  'as',
  'it',
  'will',
  'be',
  '.']]

#### E os stems? São diferentes?

O stemmer é um algoritmo para remover terminações flexionais e derivacionais, a fim de reduzir as formas de palavras a um radical comum.

In [10]:
[content['stem'] for content in result]

[['lifechang',
  'it',
  'i',
  'chang',
  ',',
  'continu',
  'chang',
  ',',
  'inevit',
  'chang',
  ',',
  'that',
  'i',
  'the',
  'domin',
  'factor',
  'in',
  'societi',
  'todai',
  '.',
  'no',
  'sensibl',
  'decis',
  'can',
  'be',
  'made',
  'ani',
  'longer',
  'without',
  'take',
  'into',
  'account',
  'not',
  'onli',
  'the',
  'world',
  'a',
  'it',
  'i',
  ',',
  'but',
  'the',
  'world',
  'a',
  'it',
  'will',
  'be',
  '.']]

# Part of Speech (POS)

![pos](https://miro.medium.com/max/1170/1*CbZE2ZTBlmswW84Knjbqkg.png)

* `CC` coordinating conjunction
* `CD` cardinal digit
* `DT` determiner
* `EX` existential there (like: “there is” … think of it like “there exists”)
* `FW` foreign word
* `IN` preposition/subordinating conjunction
* `JJ` adjective ‘big’
* `JJR` adjective, comparative ‘bigger’
* `JJS` adjective, superlative ‘biggest’
* `LS` list marker 1)
* `MD` modal could, will
* `NN` noun, singular ‘desk’
* `NNS` noun plural ‘desks’
* `NNP` proper noun, singular ‘Harrison’
* `NNPS` proper noun, plural ‘Americans’
* `PDT` predeterminer ‘all the kids’
* `POS` possessive ending parent’s
* `PRP` personal pronoun I, he, she
* `PRP` possessive pronoun my, his, hers
* `RB` adverb very, silently,
* `RBR` adverb, comparative better
* `RBS` adverb, superlative best
* `RP` particle give up
* `TO`, to go ‘to’ the store.
* `UH` interjection, errrrrrrrm
* `VB` verb, base form take
* `VBD` verb, past tense took
* `VBG` verb, gerund/present participle taking
* `VBN` verb, past participle taken
* `VBP` verb, sing. present, non-3d take
* `VBZ` verb, 3rd person sing. present takes
* `WDT` wh-determiner which
* `WP` wh-pronoun who, what
* `WP` possessive wh-pronoun whose
* `WRB` wh-abverb where, when

Agora que você já entendeu o processamento que o Spark NLP realiza, vamos brincar com algumas ferramentas. Uma delas é o POS, ou Part of Speech, que classifica a categoria de cada token.

In [11]:
pos = [content['pos'] for content in result]
token = [content['token'] for content in result]
# vamos juntar token e tag para observá-los lado a lado
list(zip(token[0], pos[0]))

[('life-changing', 'VBG'),
 ('It', 'PRP'),
 ('is', 'VBZ'),
 ('change', 'NN'),
 (',', ','),
 ('continuing', 'VBG'),
 ('change', 'NN'),
 (',', ','),
 ('inivitable', 'JJ'),
 ('change', 'NN'),
 (',', ','),
 ('that', 'WDT'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('dominant', 'JJ'),
 ('factor', 'NN'),
 ('in', 'IN'),
 ('society', 'NN'),
 ('today', 'NN'),
 ('.', '.'),
 ('No', 'DT'),
 ('sensinble', 'JJ'),
 ('decision', 'NN'),
 ('can', 'MD'),
 ('be', 'VB'),
 ('made', 'VBN'),
 ('any', 'DT'),
 ('longer', 'RBR'),
 ('without', 'IN'),
 ('taking', 'VBG'),
 ('into', 'IN'),
 ('account', 'NN'),
 ('not', 'RB'),
 ('only', 'RB'),
 ('the', 'DT'),
 ('world', 'NN'),
 ('as', 'IN'),
 ('it', 'PRP'),
 ('is', 'VBZ'),
 (',', ','),
 ('but', 'CC'),
 ('the', 'DT'),
 ('world', 'NN'),
 ('as', 'IN'),
 ('it', 'PRP'),
 ('will', 'MD'),
 ('be', 'VB'),
 ('.', '.')]

# Entity Recognizer

![ent](https://miro.medium.com/max/1200/1*DED1LGhEok0s2mUUIFbv0A.jpeg)

A função do `Entity Recognizer` é localizar e classificar as menções de entidades nomeadas em textos não estruturado em categorias predefinidas, como nomes de pessoas, organizações, locais, expressões de tempo, quantidades, entre outros - dependendo da pipeline utilizada. Vamos ver como funciona!

In [30]:
pipeline = PretrainedPipeline('recognize_entities_dl', lang='en')

recognize_entities_dl download started this may take some time.
Approx size to download 157,9 MB
[OK!]


In [41]:
text = "The Mona Lisa is a 16th century oil painting created by Leonardo. It's held at the Louvre in Paris."
result = pipeline.annotate(text)

In [42]:
list(result.keys())

['entities', 'document', 'token', 'ner', 'embeddings', 'sentence']

In [43]:
result['sentence']

['The Mona Lisa is a 16th century oil painting created by Leonardo.',
 "It's held at the Louvre in Paris."]

In [44]:
result['token']

['The',
 'Mona',
 'Lisa',
 'is',
 'a',
 '16th',
 'century',
 'oil',
 'painting',
 'created',
 'by',
 'Leonardo',
 '.',
 "It's",
 'held',
 'at',
 'the',
 'Louvre',
 'in',
 'Paris',
 '.']

In [45]:
list(zip(result['token'], result['ner']))

[('The', 'O'),
 ('Mona', 'I-PER'),
 ('Lisa', 'I-PER'),
 ('is', 'O'),
 ('a', 'O'),
 ('16th', 'O'),
 ('century', 'O'),
 ('oil', 'O'),
 ('painting', 'O'),
 ('created', 'O'),
 ('by', 'O'),
 ('Leonardo', 'I-PER'),
 ('.', 'O'),
 ("It's", 'I-ORG'),
 ('held', 'O'),
 ('at', 'O'),
 ('the', 'O'),
 ('Louvre', 'I-LOC'),
 ('in', 'O'),
 ('Paris', 'I-LOC'),
 ('.', 'O')]

In [46]:
result['entities']

['Mona Lisa', 'Leonardo', "It's", 'Louvre', 'Paris']

# Sentiment analysis

![sen](https://cdn-images-1.medium.com/max/800/1*NF6AdPk6sOMNNbQE5glvEQ.png)

Spark NLP vem com dois tipos de Sentiment analysis:

*  Sentiment Detector: define um grupo de tokens como positivos, negativos, incremento ou decremento
*  Vivekn Sentiment Approach: treina um modelo ML com um sete de exemplos positivos e negativos

Ambos os tipos são treináveis para mais emoções.

In [48]:
pipeline = PretrainedPipeline('analyze_sentiment', 'en') 

analyze_sentiment download started this may take some time.
Approx size to download 4,9 MB
[OK!]


In [54]:
text = 'I hate the movie. Even if the landscapes were absolutely stunning, the writing was very poor and it made the movie so much worse. I wouldnt recomment it.'

In [55]:
result = pipeline.annotate(text)

In [56]:
print(result['sentiment'])

['negative', 'positive', 'negative']


# Aplicações

Uma maneira de entender o potencial de qualquer ferramenta é ver as aplicações que a utilizam. Um exemplo interessante do uso do Spark NLP é o projeto apresentado por Maziyar Pahani, que usou desse recurso para processar o "Relatório da Investigação de Interferência Russa na Eleição Presidencial Estadunidense de 2016", mais comumente conhecido como `relatório Muller`.

O autor do projeto o divide em três etapas:
* Primeira etapa: usar as pipelines e modelos pré treinados do Spark NLP para fazer o processamento do Relatório
* Segunda etapa: usar os modelos treinados por BERT, treinando um modelo POS tagger com Spark NLP, limpeza de dados e extração de palavras chaves e frases por POS e NER chunking
* Terceira etapa: a partir de algoritmos de grafos do GraphFrames, fazer clustering por topic modelings do Spark ML. A visualização da rede é oferecida por Gephi.

Nesse link o autor possui uma explicação extensa sobre o seu processo, abordando diversos dos mesmos tópicos tratados neste tutorial:

[https://hackernoon.com/mueller-report-for-nerds-spark-meets-nlp-with-tensorflow-and-bert-part-1-32490a8f8f12]

As imagens incluídas são alguns os resultados interessantes obtidos por Maziyar.


![app](https://hackernoon.com/photos/JTw2M3rQabaxNg3EFoNIxjmC1ZB3-r9230p2)

![trump](https://hackernoon.com/photos/JTw2M3rQabaxNg3EFoNIxjmC1ZB3-ug383x47)

![insper](https://3.bp.blogspot.com/-fvRAgvDodjY/T8SgztDikmI/AAAAAAAAAww/SaYB30Oog6I/s1600/insper.jpg)

# Crédito das Imagens 

* https://dv-website.s3.amazonaws.com/uploads/2018/11/what_is_NLP.jpg
* https://blog.dominodatalab.com/wp-content/uploads/2019/04/Pacific-AI-Table-1-cx-Sep-2019.png
* https://miro.medium.com/max/1170/1*CbZE2ZTBlmswW84Knjbqkg.png
* https://miro.medium.com/max/1200/1*DED1LGhEok0s2mUUIFbv0A.jpeg
* https://cdn-images-1.medium.com/max/800/1*NF6AdPk6sOMNNbQE5glvEQ.png
* https://hackernoon.com/photos/JTw2M3rQabaxNg3EFoNIxjmC1ZB3-r9230p2